In [3]:
import pandas as pd
import boto3
import logging

# The maximum number of stocks to scrape
MAX_STOCKS = 1000

# define a boto resource in the ohio region
dynamodb = boto3.resource('dynamodb', region_name='us-east-2')
table = dynamodb.Table('StockSentiment')
# keep scanning until we have all the data in the table
response = table.scan()
data = response['Items']
while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])
# convert the data to a pandas dataframe
sentiment_ticker_list = pd.DataFrame(data)
# convert Date column to datetime
sentiment_ticker_list['Date'] = pd.to_datetime(sentiment_ticker_list['Date'])
# make the index the Date column
sentiment_ticker_list = sentiment_ticker_list.set_index('Date').sort_index(ascending=False)

# get a list of tickers sorted by frequency, and append googl since theres a bug with alpha vantage
sorted_tickers = sentiment_ticker_list['Stock'].value_counts().index.tolist() + ['GOOGL']
# remove any stocks that contain crypto or forex and take the first MAX_STOCKS
sorted_tickers = [t for t in sorted_tickers if 'crypto' not in t.lower() and 'forex' not in t.lower()][:MAX_STOCKS]

print(sorted_tickers)

['META', 'JPM', 'SBNY', 'GS', 'GOOG', 'TSLA', 'SIVB', 'C', 'MS', 'MSFT', 'BAC', 'FRC', 'WFC', 'BCS', 'BRK-A', 'NVDA', 'AAPL', 'COIN', 'CURN', 'WMT', 'SZIHF', 'SPGI', 'MCO', 'SI-P-A', 'BABA', 'BLK', 'MALRF', 'AWON', 'WBD', 'DSSMY', 'AMZN', 'SCHW', 'PACW', 'NFLX', 'PFE', 'SPXCF', 'IVZ', 'STT', 'EVR', 'KO', 'UBS', 'COST', 'APG', 'HD', 'CS', 'AMD', 'NKE', 'NYT', 'ROKU', 'PANW', 'MRNA', 'BA', 'TCTZF', 'RBCPF', 'ABNB', 'WAL', 'ETSY', 'TOL', 'CCORF', 'AXP', 'UBER', 'ADBE', 'BYND', 'RIVN', 'MORN', 'SSNLF', 'ORCL', 'GM', 'BIDU', 'KEY', 'SNY', 'RCKY', 'TGT', 'PYPL', 'SHOP', 'JNJ', 'BX', 'LDNXF', 'SBUX', 'NWG', 'FSLR', 'TFC', 'BNS', 'GME', 'ULTA', 'ABBV', 'AMAT', 'AVGO', 'GETY', 'MDNDF', 'ATVI', 'ZS', 'UAL', 'FDX', 'LI', 'ENVX', 'RYDAF', 'MA', 'PM', 'XPEV', 'OXY', 'PDD', 'INTC', 'CRWD', 'LEN', 'F', 'III', 'NIO', 'O', 'FRBA', 'NTES', 'GOLD', 'ISPO', 'DKS', 'M', 'SNOW', 'CVS', 'FIVE', 'MCQEF', 'CTLT', 'APE', 'PARA', 'NAUBF', 'DOCU', 'ON', 'DASH', 'BBBY', 'MELI', 'TTD', 'SSTK', 'CSCO', 'XOM', 'SCBFF

In [4]:
sorted_tickers

['META',
 'JPM',
 'SBNY',
 'GS',
 'GOOG',
 'TSLA',
 'SIVB',
 'C',
 'MS',
 'MSFT',
 'BAC',
 'FRC',
 'WFC',
 'BCS',
 'BRK-A',
 'NVDA',
 'AAPL',
 'COIN',
 'CURN',
 'WMT',
 'SZIHF',
 'SPGI',
 'MCO',
 'SI-P-A',
 'BABA',
 'BLK',
 'MALRF',
 'AWON',
 'WBD',
 'DSSMY',
 'AMZN',
 'SCHW',
 'PACW',
 'NFLX',
 'PFE',
 'SPXCF',
 'IVZ',
 'STT',
 'EVR',
 'KO',
 'UBS',
 'COST',
 'APG',
 'HD',
 'CS',
 'AMD',
 'NKE',
 'NYT',
 'ROKU',
 'PANW',
 'MRNA',
 'BA',
 'TCTZF',
 'RBCPF',
 'ABNB',
 'WAL',
 'ETSY',
 'TOL',
 'CCORF',
 'AXP',
 'UBER',
 'ADBE',
 'BYND',
 'RIVN',
 'MORN',
 'SSNLF',
 'ORCL',
 'GM',
 'BIDU',
 'KEY',
 'SNY',
 'RCKY',
 'TGT',
 'PYPL',
 'SHOP',
 'JNJ',
 'BX',
 'LDNXF',
 'SBUX',
 'NWG',
 'FSLR',
 'TFC',
 'BNS',
 'GME',
 'ULTA',
 'ABBV',
 'AMAT',
 'AVGO',
 'GETY',
 'MDNDF',
 'ATVI',
 'ZS',
 'UAL',
 'FDX',
 'LI',
 'ENVX',
 'RYDAF',
 'MA',
 'PM',
 'XPEV',
 'OXY',
 'PDD',
 'INTC',
 'CRWD',
 'LEN',
 'F',
 'III',
 'NIO',
 'O',
 'FRBA',
 'NTES',
 'GOLD',
 'ISPO',
 'DKS',
 'M',
 'SNOW',
 'CVS',
 'FIVE'

In [20]:
json.dumps(sorted_tickers[6:10])

'["SIVB", "C", "MS", "MSFT"]'

In [23]:
import requests
import json
url = 'https://ddcp87x7x1.execute-api.us-east-2.amazonaws.com/beta/scrape/prices'
data = json.dumps(sorted_tickers[5:10])
response = requests.post(url, data=data)
print(response.text)

"successful"


In [11]:
response.content

b'{"message": "Endpoint request timed out"}'

In [30]:
import threading
import requests
import json
from tqdm import tqdm

def request_task(url, json_data):
    response = requests.post(url, json=json_data, timeout=600)
    print(response.text)


def fire_and_forget(url, json_data):
    threading.Thread(target=request_task, args=(url, json_data)).start()

url = 'https://ddcp87x7x1.execute-api.us-east-2.amazonaws.com/beta/scrape/prices'
for i in tqdm(range(0, len(sorted_tickers), 5)):
    data = json.dumps(sorted_tickers[i:i+5])
    fire_and_forget(url, json_data=data)

fire_and_forget(url, json_data=json.dumps(['GOOGL']))

100%|██████████| 201/201 [00:00<00:00, 509.50it/s] 


In [27]:
sorted_tickers[i:]

['GOOGL']